# Transformer

In [ ]:
vedi appunti x teoria
vedi Lezione6 -> Attention

In [1]:
from keras.layers import Attention

2023-09-25 10:15:46.615965: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 10:15:47.054130: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 10:15:47.057485: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 10:15:49.234933: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## BERT -- Encoder

In [ ]:
vedi appunti x teoria

## Hugging Face
vedi file prof. "huggingface"

+ repository di modelli (checkpoints)
+ vari task (NLP, ComputerVision, ...)
+ tanti dataset
+ librerie di HF
+ https://huggingface.co/docs/transformers/index

### Librerie

In [2]:
!pip install tokenizers transformers datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow>=8.0.0 from https://files.pythonhosted.org/packages/49/db/0a40d2a5b2382c77536479894ce2900e5f4c40251681a72d397ba6430f8d/pyarrow-13.0.0-cp39-cp39-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/45/63/40da996350689cf29db7f8819aafa74c9d36feca4f0e4393d220c619a1dc/xxhash-3.3.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/c6/c9/820b5ab056f4ada76fbe05bd481a948f287957d6cbfd59e2dd2618b408c1/multiprocess-0.70.15-py39-none-any.whl.metadata
  Obtaining dependency information for dill<0.3.8,>=0.3.0 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdf

In [1]:
from transformers import pipeline
from tensorflow import keras
import keras

/home/naska/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-25 17:07:14.008478: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 17:07:16.034117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Pipeline - Task di Question&Answer

In [2]:
pipe = pipeline(task = "question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [5]:
pipe(context = "Rizzoli is a school based in Milan", question = "Where is Rizzoli?")

{'score': 0.9945299029350281, 'start': 29, 'end': 34, 'answer': 'Milan'}

### Task di Sentiment Analysis

In [7]:
import datasets

In [8]:
dataset = datasets.load_dataset(path = "emotion")

In [9]:
dataset["validation"] # i dataset sono caricati un pezzo alla volta, o anche in streaming da online

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [10]:
dataset["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [11]:
train_dataset = dataset["train"]
train_dataset.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [12]:
train_dataset.features["label"]

ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)

In [13]:
train_dataset[:4] # estrae 4 elementi, 4 features e 4 labels

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property'],
 'label': [0, 0, 3, 2]}

In [14]:
dataset.set_format("pandas") # è un'opzione
train_dataset[:4]

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2


In [15]:
dataset.reset_format()

In [16]:
# label_descr: nuova colonna del dataset 
# dataset.map() prende un sample e lo trasforma
# in HF un sample è sempre un dict. -- text : label

# nuovo dataset
int2str = train_dataset.features["label"].int2str
lambda x: dict( label_descr = int2str(x["label"]) )

dataset = dataset.map(
    lambda x: dict( label_descr = train_dataset.features["label"].int2str(x["label"]) ) 
)

Nota: {a : 1} è stesso che dict(a=1)

Map: 100%|█████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 14936.16 examples/s]


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_descr'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_descr'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_descr'],
        num_rows: 2000
    })
})

In [18]:
dataset.set_format("pandas")
dataset["train"][:5]

,text,label,label_descr
0,i didnt feel humiliated,0,sadness
1,i can go from feeling so hopeless to so damned...,0,sadness
2,im grabbing a minute to post i feel greedy wrong,3,anger
3,i am ever feeling nostalgic about the fireplac...,2,love
4,i am feeling grouchy,3,anger


### Classificazione con Bert: Sentiment Analysis
modello pre-addestrato e poi fine-tuning

In [22]:
from transformers import pipeline, AutoConfig, AutoModel, TFAutoModel, AutoTokenizer 
# TFAutomodel per scaricare la versione tensorflow dello stesso modello

In [20]:
model = TFAutoModel.from_pretrained("bert-base-uncased")
# Nota: scarica architettura + pesi

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [21]:
model.framework

'tf'

In [24]:
model

In [25]:
isinstance(model, keras.Model) # il modello è una sottoclasse di un modello di Keras

True

In [27]:
model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


In [28]:
model_pt = AutoModel.from_pretrained("bert-base-uncased")
model_pt # outputs l'architettura

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

Nota:
il modello è composto da due parti
+ body: modello base
+ head: aggiunta, dipende dal task

l'**head** è la parte da allenare per il nostro task

In [29]:
from transformers import TFAutoModelForSequenceClassification, AutoModelForSequenceClassification

In [30]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# scarica il body di BERT, allenato per il suo task originale, e poi adds a head for SequenceClassification, che è da allenare
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [31]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") 
# Nota: deve ess. lo stesso Tokenizzatore usato in fase di Training
# ogni Checkpoint ha il suo tokenizzatore -> download con AutoTokenizer

In [32]:
tokenizer(
    [
     "I'm very good at english", 
     "I'm here"
    ]
)

{'input_ids': [[101, 1045, 1005, 1049, 2200, 2204, 2012, 2394, 102], [101, 1045, 1005, 1049, 2182, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

**Nota**
+ il tokenizer fa un padding delle parole più corte: attention_mask, se ha degli zeri -> c'è padding
+ token_type_ids

In [35]:
encoded = tokenizer("I'm very good at english")
print(encoded)

tokens = tokenizer.decode(encoded.input_ids)
print(tokens)

{'input_ids': [101, 1045, 1005, 1049, 2200, 2204, 2012, 2394, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] i'm very good at english [SEP]


In [36]:
encoded = tokenizer("I'm very good at english", "I'm not good in spanish")
encoded # una coppia di frasi x sentence similarity (se le 2 frasi sono consequenziali)

{'input_ids': [101, 1045, 1005, 1049, 2200, 2204, 2012, 2394, 102, 1045, 1005, 1049, 2025, 2204, 1999, 3009, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [38]:
config = AutoConfig.from_pretrained("bert-base-uncased")
config # parametri

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

#### Encoding del Testo

In [39]:
tokenizer("I'm okay", "yesterday")
# token_type_ids si capisce meglio tokenizzando una coppia di frasi

{'input_ids': [101, 1045, 1005, 1049, 3100, 102, 7483, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [40]:
tokenizer.model_input_names

['input_ids', 'token_type_ids', 'attention_mask']

In [41]:
dataset.reset_format()

dataset = dataset.map(
    lambda x: tokenizer( x["text"] )
)

Map: 100%|██████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 7100.83 examples/s]


In [42]:
dataset.set_format("pandas")
dataset["train"][:2]

,text,label,label_descr,input_ids,token_type_ids,attention_mask
0,i didnt feel humiliated,0,sadness,"[101, 1045, 2134, 2102, 2514, 26608, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
1,i can go from feeling so hopeless to so damned...,0,sadness,"[101, 1045, 2064, 2175, 2013, 3110, 2061, 2062...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [43]:
trainset = dataset["train"]
trainset = trainset.remove_columns(["text", "label_descr"])
trainset[:2]

,label,input_ids,token_type_ids,attention_mask
0,0,"[101, 1045, 2134, 2102, 2514, 26608, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
1,0,"[101, 1045, 2064, 2175, 2013, 3110, 2061, 2062...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [44]:
import numpy as np

In [45]:
def collate_padding(batch):  # un batch è un dictionary. Keys: Values. label: [...] input_ids: [[...][...]]
    # la funzione fa padding degli input_ids
    max_len = max([len(seq["input_ids"]) for seq in batch])
    def pad(l):
        return np.concatenate([l, [0]*(max_len-len(l))])

    d = {}
    d["label"] = [el["label"] for el in batch]
    for key in ['input_ids', 'token_type_ids', 'attention_mask']:
        d[key] = [pad(l[key]) for l in batch]

    return d

In [46]:
help(trainset.to_tf_dataset)

Help on method to_tf_dataset in module datasets.arrow_dataset:

to_tf_dataset(batch_size: Optional[int] = None, columns: Union[str, List[str], NoneType] = None, shuffle: bool = False, collate_fn: Optional[Callable] = None, drop_remainder: bool = False, collate_fn_args: Optional[Dict[str, Any]] = None, label_cols: Union[str, List[str], NoneType] = None, prefetch: bool = True, num_workers: int = 0, num_test_batches: int = 20) method of datasets.arrow_dataset.Dataset instance
    Create a `tf.data.Dataset` from the underlying Dataset. This `tf.data.Dataset` will load and collate batches from
    the Dataset, and is suitable for passing to methods like `model.fit()` or `model.predict()`. The dataset will yield
    `dicts` for both inputs and labels unless the `dict` would contain only a single key, in which case a raw
    `tf.Tensor` is yielded instead.
    
    Args:
        batch_size (`int`, *optional*):
            Size of batches to load from the dataset. Defaults to `None`, which imp

In [47]:
# Trasforma in Dataset Tensorflow

trainset_tf = trainset.to_tf_dataset(
    collate_fn=collate_padding, # funzione che costruisce batch da singoli sample
    batch_size=32, label_cols="label", shuffle=True, columns=['input_ids', 'token_type_ids', 'attention_mask'])

In [48]:
next(iter(trainset_tf))

({'input_ids': <tf.Tensor: shape=(32, 40), dtype=float32, numpy=
  array([[  101.,  1045.,  2018., ...,     0.,     0.,     0.],
         [  101.,  1045.,  2514., ...,     0.,     0.,     0.],
         [  101.,  1045.,  2123., ...,     0.,     0.,     0.],
         ...,
         [  101.,  1045.,  2228., ...,     0.,     0.,     0.],
         [  101., 10047.,  2145., ...,     0.,     0.,     0.],
         [  101.,  5665.,  2022., ...,     0.,     0.,     0.]],
        dtype=float32)>,
  'token_type_ids': <tf.Tensor: shape=(32, 40), dtype=float32, numpy=
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
  'attention_mask': <tf.Tensor: shape=(32, 40), dtype=float32, numpy=
  array([[1., 1., 1., ..., 0., 0., 0.],
         [1., 1., 1., ..., 0., 0., 0.],
         [1., 1., 1

In [49]:
model.compile(keras.optimizers.Adam(1e-5))

AttributeError: 'BertForSequenceClassification' object has no attribute 'compile'

## HF file Prof.

In [ ]:
# Sentiment analysis

dataset = datasets.load_dataset(path="emotion")

dataset

dataset["train"][0]

train_dataset = dataset["train"]

train_dataset.features

train_dataset[:4]

In [ ]:
dataset.set_format("pandas")

train_dataset[:4]

dataset.reset_format()

# label_str
dataset = dataset.map(lambda x: dict(label_descr=train_dataset.features["label"].int2str(x["label"])))

dataset.set_format("pandas")

dataset["train"][:5]

model = TFAutoModel.from_pretrained("distilbert-base-uncased")

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

encoded = tokenizer("I'm very good at english", "I'm not good in spanish")

encoded

tokens = tokenizer.decode(encoded.input_ids)

tokens

config = AutoConfig.from_pretrained("distilbert-base-uncased")

config

dataset["train"][:2]

tokenizer("I'm ok", "yesterday")

tokenizer.model_input_names

dataset["train"][:2]

dataset.reset_format()

dataset = dataset.map(lambda x: tokenizer(x["text"]))

trainset[:2]

dataset.set_format("pandas")

trainset = dataset["train"]

trainset = trainset.remove_columns(["text", "label_descr"])

trainset[:2]

list(trainset.features.keys())

trainset[:4].keys()

In [ ]:
import numpy as np

def collate_padding(batch): # prende un set di sample e li trasforma in un batch
    # batch = lista di record (come dizionari)
    max_len = max([len(seq["input_ids"]) for seq in batch])
    def pad(l):
        return np.concatenate([l, [0]*(max_len-len(l))])

    d = {}
    d["label"] = [el["label"] for el in batch]
    for key in ['input_ids', 'attention_mask']:
        d[key] = [pad(l[key]).astype(np.int32) for l in batch]

    return d

In [ ]:
collate_padding([dataset["train"][i] for i in range(2)]) # nota la attention mask delle due frasi (con gli zeri: c'è padding)

In [ ]:
help(trainset.to_tf_dataset)

tokenizer.model_input_names

In [ ]:
trainset_tf = dataset["train"].to_tf_dataset(
    collate_fn=collate_padding, 
    batch_size=32, label_cols="label", shuffle=True, columns=['input_ids', 'attention_mask'])

In [ ]:
valset_tf = dataset["validation"].to_tf_dataset(
    collate_fn=collate_padding, 
    batch_size=32, label_cols="label", shuffle=True, columns=['input_ids', 'attention_mask'])

In [ ]:
testset_tf = dataset["test"].to_tf_dataset(
    collate_fn=collate_padding, 
    batch_size=32, label_cols="label", shuffle=False, columns=['input_ids', 'attention_mask'])

**Nota**: classi per caricare/usare dataset (i dataset vengono caricati dinamicamente)
+ tf.data.Dataset
+ keras.Sequence
+ in alternativa: fare un generatore

  

In [ ]:
next(iter(trainset_tf))

# in alternativa
for b in trainset_tf:
    print(b)
    break

In [ ]:
model.compile(keras.optimizers.Adam(1e-5), metrics=["accuracy"])
# learning rate basso perché è un finetuning, fare aggiustamenti piccoli. altrimenti modifica il body di BERT che si vuole invece preservare

In [ ]:
model.fit(x=trainset_tf, validation_data=valset_tf, epochs=1) # validation_split=0.2 si può fare solo se passo tensori. con dataset e generatori no, x=trainset contiene già feature e label
# però gli passo il validation_data: valset_tf

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

In [ ]:
preds = model.predict(testset_tf, batch_size = 32)
preds.logits

In [ ]:
preds.logits.shape

In [ ]:
predicted_classes = np.array(preds.logits, axis=-1)

In [ ]:
predicted_classes

In [ ]:
real_classes = np.array(dataset["test"]["label"])

In [ ]:
# calcolo accuracy
accuracy_score(real_classes, predicted_classes)

In [ ]:
labels=dataset["train"].features["label"].names

In [ ]:
cm = confusion_matrix(real_classes, predicted_classes)

In [ ]:
ConfusionMatrixDisplay(cm, )